# Lab 6

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
KS = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/kevin.xlsx',
                  header=1)

DI = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/Daniel.xlsx',
                  header=13, index_col=1)

TC = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/Tyler H.xlsx',
                  header=1)

## Clean up and analyze data

In [3]:
def abs_ave(D, x0=1, xf=9, dx=10, y0=0, yf=4, n=9, Nanx=[], Nany=[], 
            Inh="Name IDIOT!!!", Unit="Units DUMBASS!!!", 
            ONPG = [10, 20, 40, 80], I=[0, 10, 20, 40], C=1, V=200, time=4):
    # 2D data size
    x = xf-x0
    y = yf-y0
    
    # Turn into 3d array
    # Axis 1 = Substrate
    # Axis 2 = Inhibitor
    # Axis 3 = Time
    A = np.zeros((x, y, n))
    for i in range(0,n):
        a = np.array(D.iloc[x0+dx*i:xf+dx*i, y0:yf])
        A[:,:,i] = a 
    
    # Find average change in absorbance with time
    Diff = np.diff(A, axis=2)
    Ave = np.nanmean(Diff, axis=2)
    Ave = pd.DataFrame(Ave)
    
    # Get rid of errors
    if len(Nanx)>0:
        for i in range(0, len(Nanx)):
            Ave.iloc[Nanx[i], Nany[i]] = np.nan

    else: 
        Ave += 0
        
    # Get averages of change with respect to time
    Abs = np.zeros((int(x/2), y))
    for i in range(0, int(x/2)):
        Ave_abs = np.nanmean(Ave.iloc[2*i:2*i+2,:], axis=0)
        Abs[i,:] = Ave_abs

    # Label columns
    Abs = pd.DataFrame(Abs)
    I_c = [i*C/V for i in I]
    Abs.columns = [r'{0}: {1:#.3g} {2}'.format(Inh, i, Unit) for i in I_c]
    ABS = Abs.T
    ABS.columns = [i * 4/V for i in ONPG]
    return ABS.T*time, Ave*time

In [4]:
KS_abs, KS_ave = abs_ave(D=KS, Nanx = [0, 7, 4], Nany = [0, 0, 1], Inh='[Sucrose]', Unit='mmoles/l')
TC_abs, TC_ave = abs_ave(D=TC, Nanx = [1, 3], Nany = [0, 1], Inh='[Calcium]', C=200, Unit='g/l')
DI_abs, DI_ave = abs_ave(D=DI, Nanx = [3, 4], Nany = [3, 3], x0=1, xf=9, y0=1, yf=5, dx=12, Inh='[IPTG]', C=2, Unit='g/l')

In [5]:
# KS_abs

In [6]:
# TC_abs

In [7]:
# DI_abs

## Functions to generate graphs

In [8]:
P = ['xkcd:lightish green', 
     'xkcd:medium green', 
     'xkcd:jungle green', 
     'xkcd:deep green']

# Fit enzyme
def enz_func(x, a, b, c):
    y = a*(c - np.exp(-b*x))
    return y

def curve_fit(f, x, D, Colour=P, append=False):
    x = np.array(x)
    if append == True: 
        X = np.append(x, 0)
    else: 
        X = x
    xmax = np.max(x)
    xmin = np.min(x)
    xrange = np.linspace(0, xmax, 100)
    Popt = []
    for i in range(0, np.shape(D)[0]):
        y = np.array(D.iloc[:, i])
        if append == True: 
            Y = np.append(y, 0)
        else: 
            Y = y
        popt, pcov = sci.optimize.curve_fit(f, X, Y, method='trf')
        Popt.append(popt)
        plt.plot(xrange, f(xrange, *popt), c=Colour[i])
    return Popt

# Normal plot 
def plot_norm(D, f, xname='[ONPG] (mg/ml)', yname=r'$v_0$ ($A_{405}$)', 
              plot_fit=False, Colour=P, append=False):
    plt.figure(figsize=(10, 5))
    Lines = sns.scatterplot(data=D,
                         markers=['o', 'o', 'o', 'o'], 
                         palette=Colour)
    
    A = D.T
    A_name = list(A.columns.values)
    
    if plot_fit == True:
        Popt = curve_fit(f, x=A_name, D=D, append=append)
    
    plt.xlabel(xname, fontsize=12)
    plt.ylabel(yname, fontsize=12)
    plt.xlim()
    plt.legend(loc=4)
    plt.grid()
    
# Lineweaver-Burk plot
def plot_LB(D, xname=r'$[ONPG]^{-1}$ (ml/mg)', 
            yname=r'$v_0^{-1}$ $\left(A_{405}^{-1}\right)$', Colour=P):
    
    A = D.T
    A_name = list(A.columns.values)
    D_name = [1/i for i in A_name]
    A.columns = D_name
    D = 1/(A.T)
    
    palette=Colour
    
    plot_norm(D, xname, yname, plot_fit=False)
    xInts = []
    yInts = []
    for i in range(0, np.shape(D)[1]):
        slope, intercept, r_value, p_value, std_err = stats.linregress(D_name, D.iloc[:,i]) 
#         xmin = -intercept/slope
        xmin = np.min(D_name)
        xmax = np.max(D_name)
        xrange = np.linspace(xmin, xmax, 100)
        y = slope*xrange + intercept
        xInts.append(-intercept/slope)
        yInts.append(intercept)
        plt.plot(xrange, y, color=palette[i])
    
    D_col_name = list(D.columns.values)
    V_max = [1/i for i in yInts]
    K_m = [-1/i for i in xInts]
    legend_label = [r'{0}; $V_{{Max}}=${1:#.3G}; $K_M=${2:#.3G}'.format(D_col_name[i], 
                    V_max[i], K_m[i]) for i in range(0, len(K_m))]
    plt.legend(legend_label)
#     plt.xlim(right=np.max(D_name)+2)
    return D

In [9]:
# KS_LB

In [10]:
# Plot it out 
# ===========

# Note: V_max between graphs inconsintent 
# append=True makes the curve start from (0, 0)
plot_norm(KS_abs, enz_func, plot_fit=True, append=False)
plot_norm(TC_abs, enz_func, plot_fit=True, append=False)
plot_norm(DI_abs, enz_func, plot_fit=True, append=False)
plt.show()

KS_LB = plot_LB(KS_abs)
TC_LB = plot_LB(TC_abs)
DI_LB = plot_LB(DI_abs)

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>